## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-02-15-02-53 +0000,bbc,Captain guilty of North Sea tanker crash death,https://www.bbc.com/news/articles/c9dv240w0w3o...
1,2026-02-02-15-01-00 +0000,wsj,Oil Futures Ease Amid U.S.-Iran De-Escalation,https://www.wsj.com/finance/commodities-future...
2,2026-02-02-14-58-27 +0000,nyt,How the Supreme Court Secretly Made Itself Eve...,https://www.nytimes.com/2026/02/02/us/supreme-...
3,2026-02-02-14-58-00 +0000,wsj,"Disney Shares Fall Despite Streaming Growth, a...",https://www.wsj.com/business/media/disney-dis-...
4,2026-02-02-14-57-58 +0000,nypost,Wild moment robbers smash jewelery store windo...,https://nypost.com/2026/02/02/world-news/wild-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2449/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
54,trump,29
252,epstein,18
188,new,17
141,ice,17
166,grammy,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
213,2026-02-01-22-34-48 +0000,nypost,India to buy oil from Venezuela rather than Ru...,https://nypost.com/2026/02/01/world-news/india...,73
58,2026-02-02-12-44-00 +0000,wsj,President Trump says the Kennedy Center will b...,https://www.wsj.com/politics/elections/trump-s...,70
17,2026-02-02-14-37-23 +0000,nyt,Trump Says Kennedy Center Will Close for 2-Yea...,https://www.nytimes.com/2026/02/01/us/politics...,70
146,2026-02-02-07-37-00 +0000,nypost,Trump threatens to sue Trevor Noah after Gramm...,https://nypost.com/2026/02/02/us-news/donald-t...,69
172,2026-02-02-04-05-18 +0000,latimes,Trump says 'dilapidated' Kennedy Center will c...,https://www.latimes.com/world-nation/story/202...,62


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
213,73,2026-02-01-22-34-48 +0000,nypost,India to buy oil from Venezuela rather than Ru...,https://nypost.com/2026/02/01/world-news/india...
176,50,2026-02-02-03-38-34 +0000,startribune,The latest: Musicians call out ICE actions in ...,https://www.startribune.com/ice-raids-minnesot...
218,47,2026-02-01-22-15-31 +0000,bbc,'Do you think you're the devil himself?' Epste...,https://www.bbc.com/news/videos/cx20zqw7v9ko?a...
58,41,2026-02-02-12-44-00 +0000,wsj,President Trump says the Kennedy Center will b...,https://www.wsj.com/politics/elections/trump-s...
144,37,2026-02-02-07-53-57 +0000,bbc,Drone incidents near UK military bases double ...,https://www.bbc.com/news/articles/c23rxr1lz8do...
106,35,2026-02-02-10-51-21 +0000,bbc,Israel reopens Gaza's key Rafah border crossin...,https://www.bbc.com/news/articles/c99kdjdj9l8o...
194,31,2026-02-02-01-12-32 +0000,nypost,Ex-Gov. Chris Christie claims Trump is politic...,https://nypost.com/2026/02/01/us-news/trump-is...
29,31,2026-02-02-14-14-31 +0000,nypost,"Ken Griffin, Stephen Ross bankroll $10M campai...",https://nypost.com/2026/02/02/business/ken-gri...
158,28,2026-02-02-06-15-08 +0000,nypost,Savannah Guthrie’s mother reported missing in ...,https://nypost.com/2026/02/02/entertainment/sa...
112,26,2026-02-02-10-30-00 +0000,wsj,"Heard on the Street: Kevin Warsh, President Tr...",https://www.wsj.com/economy/central-banking/sa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
